In [9]:
import pandas as pd
from sklearn.metrics import f1_score

train_df = pd.read_csv('./dataset/train_set.csv', sep='\t', nrows=40000)
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df[['text', 'label_ft']].iloc[:-5000].to_csv('./dataset/train.csv', index=None, sep='\t', header=None)


In [8]:
"""
保持lr, wordNgrams, verbose, minCount, epoch, loss， 测试集不变，仅修改训练集大小，结果如下

15K  0.8296
30K  0.8782
50K  0.8915
"""

import fasttext

model = fasttext.train_supervised('./dataset/train.csv', lr=1, wordNgrams=2, verbose=2, minCount=1, epoch=25, loss='hs')

val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[-5000:]['text']]
print(f1_score(train_df['label'].values[-5000:].astype(str), val_pred, average='macro'))

0.8914924964163824


In [10]:
"""
5折交叉验证，感谢评论区@jackkuo同学的代码参考
本地跑不太动，没有仔细搜索超参。
"""

from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True)
max_score = 0

for train_idx, eval_idx in kf.split(train_df['text'], train_df['label_ft']):
    train_df[['text', 'label_ft']].iloc[train_idx].to_csv('./dataset/train_ft.csv', index=None, sep='\t', header=None)
    model = fasttext.train_supervised('./dataset/train_ft.csv', lr=1, wordNgrams=2, verbose=2, minCount=1, epoch=25, loss='hs')

    val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df.iloc[eval_idx]['text']]
    score = f1_score(train_df['label'].values[eval_idx].astype(str), val_pred, average='macro')
    print(score)

    if score > max_score:
        max_score = score
        model.save_model('./dataset/best_model.pkl')

print(f'max_score: {max_score}')

0.8716930385026631
0.8675029297024268
0.8761717368821362
0.8816159961572956
0.8838806134729039
max_score: 0.8838806134729039
